<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/example_kaggle/les_3/supervised_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None  # default='warn'
# це налаштування в pandas, яке керує попередженням про "chained assignment" — тобто випадки,
# коли ти змінюєш значення в DataFrame у два кроки (ланцюжком),
# що може призвести до помилок або несподіваної поведінки.

# CSV

we will use the Kaggle Food Demand Forecasting data to demonstrate these major steps.
- Data https://kaggle.com/datasets/kannanaikkal/food-demand-forecasting/

In [ ]:
# Import data from kaggle competition
PATH = "./data/food_demand"
train_data = pd.read_csv(os.path.join(PATH, "train.csv"), index_col=0)
test_data = pd.read_csv(os.path.join(PATH, "test.csv"), index_col=0)

train_data.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
id,,,,,,,,
1379560,1,55,1885,136.83,152.29,0,0,177
1466964,1,55,1993,136.83,135.83,0,0,270
1346989,1,55,2539,134.86,135.86,0,0,189
1338232,1,55,2139,339.50,437.53,0,0,54
1448490,1,55,2631,243.50,242.50,0,0,40


In [ ]:
train_data.tail()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
id,,,,,,,,
1271326,145,61,1543,484.09,484.09,0,0,68
1062036,145,61,2304,482.09,482.09,0,0,42
1110849,145,61,2664,237.68,321.07,0,0,501
1147725,145,61,2569,243.50,313.34,0,0,729
1361984,145,61,2490,292.03,290.03,0,0,162


In [ ]:
test_data.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
id,,,,,,,
1028232,146,55,1885,158.11,159.11,0,0
1127204,146,55,1993,160.11,159.11,0,0
1212707,146,55,2539,157.14,159.14,0,0
1082698,146,55,2631,162.02,162.02,0,0
1400926,146,55,1248,163.93,163.93,0,0


In [ ]:
meal_info_df = pd.read_csv(os.path.join(PATH, "meal_info.csv"), index_col=0)
meal_info_df = meal_info_df.reset_index(drop=False)
meal_info_df.tail()

,meal_id,category,cuisine
46,2956,Fish,Continental
47,2104,Fish,Continental
48,2444,Seafood,Continental
49,2867,Seafood,Continental
50,1445,Seafood,Continental


In [ ]:
meal_info_df[["category", "meal_id"]].groupby(["category"]).count()

,meal_id
category,
Beverages,12
Biryani,3
Desert,3
Extras,3
Fish,3
Other Snacks,3
Pasta,3
Pizza,3
Rice Bowl,3


In [ ]:
# train_data = train_data.merge(meal_info_df, on="meal_id", how="left")
# train_data

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category_x,cuisine_x,category_y,cuisine_y
0,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,Beverages,Thai
1,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai,Beverages,Thai
2,1,55,2539,134.86,135.86,0,0,189,Beverages,Thai,Beverages,Thai
3,1,55,2139,339.50,437.53,0,0,54,Beverages,Indian,Beverages,Indian
4,1,55,2631,243.50,242.50,0,0,40,Beverages,Indian,Beverages,Indian
...,...,...,...,...,...,...,...,...,...,...,...,...
456543,145,61,1543,484.09,484.09,0,0,68,Desert,Indian,Desert,Indian
456544,145,61,2304,482.09,482.09,0,0,42,Desert,Indian,Desert,Indian
456545,145,61,2664,237.68,321.07,0,0,501,Salad,Italian,Salad,Italian
456546,145,61,2569,243.50,313.34,0,0,729,Salad,Italian,Salad,Italian


In [ ]:
train_data = train_data.merge(meal_info_df, on="meal_id", how="inner")
train_data

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine
0,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai
1,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai
2,1,55,2539,134.86,135.86,0,0,189,Beverages,Thai
3,1,55,2139,339.50,437.53,0,0,54,Beverages,Indian
4,1,55,2631,243.50,242.50,0,0,40,Beverages,Indian
...,...,...,...,...,...,...,...,...,...,...
456543,145,61,1543,484.09,484.09,0,0,68,Desert,Indian
456544,145,61,2304,482.09,482.09,0,0,42,Desert,Indian
456545,145,61,2664,237.68,321.07,0,0,501,Salad,Italian
456546,145,61,2569,243.50,313.34,0,0,729,Salad,Italian


In [ ]:
train_data[["category", "meal_id"]].groupby(["category"]).count().sort_values(by="meal_id", ascending=False)

,meal_id
category,
Beverages,127890
Rice Bowl,33408
Sandwich,33291
Pizza,33138
Starters,29941
Other Snacks,29379
Desert,29294
Salad,28559
Pasta,27694


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   week                   456548 non-null  int64  
 1   center_id              456548 non-null  int64  
 2   meal_id                456548 non-null  int64  
 3   checkout_price         456548 non-null  float64
 4   base_price             456548 non-null  float64
 5   emailer_for_promotion  456548 non-null  int64  
 6   homepage_featured      456548 non-null  int64  
 7   num_orders             456548 non-null  int64  
 8   category               456548 non-null  object 
 9   cuisine                456548 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 34.8+ MB


🎯 Навіщо це потрібно:
Це корисна перевірка для виявлення дублікатів:

Наприклад, якщо модель тренується на тижневих замовленнях і передбачається, що кожен meal_id має лише один запис на тиждень, тоді дублікати — це проблема.

Це може вказувати на помилки у даних або специфіку бізнес-логіки (наприклад, одна страва була замовлена кілька разів у тиждень різними клієнтами — тоді треба агрегацію).

In [ ]:
for i in range(146):
    slice = train_data[train_data["week"] == i]
    length = len(slice)
    unique_ids = slice["meal_id"].unique()
    if length != len(unique_ids):
        print(f"Week {i} more than one entry per meal_id. {length}, {unique_ids }.")

Week 1 more than one entry per meal_id. 2922, [1885 1993 2539 2139 2631 1248 1778 1062 2707 1207 1230 2322 2290 1727
 1109 2640 2306 2126 2826 1754 1971 1902 1311 1803 1558 2581 1962 1445
 2444 2867 1525 2704 2304 2577 1878 1216 1247 1770 1198 1438 2494 1847
 2760 2492 1543].
Week 2 more than one entry per meal_id. 2896, [1885 1993 2539 2631 1248 1778 1062 2707 1207 1230 2322 2290 1727 1109
 2640 2306 2826 1754 1971 1311 1803 1558 2581 1962 1445 2444 2867 1525
 2704 2492 1543 2304 2139 2577 1878 2126 1902 1247 1198 1438 2494 1847
 2760 1216 1770].
Week 3 more than one entry per meal_id. 2899, [1885 1993 2539 2139 1248 1778 1062 2707 1207 1230 2322 2290 1727 1109
 2640 2306 1216 2126 2826 1754 1971 1902 1247 1311 1803 1558 2581 1962
 2444 2704 2492 1543 2631 2577 1878 1198 1438 2494 1847 1445 2760 1525
 2304 1770 2867].
Week 4 more than one entry per meal_id. 2889, [1885 1993 2539 1248 1778 1062 2707 1207 1230 2322 2290 1727 1109 2640
 2126 2826 1754 1971 1247 1311 1803 1558 2581 1962 1

🧠 Навіщо це потрібно:
🔹 Щоб перевірити:

❓ Чи є дублікати — тобто, одна і та ж страва в одному і тому ж центрі з'являється кілька разів у тому самому тижні.

🔁 Якщо таке повторення є — можливо, потрібно агрегувати замовлення по meal_id та center_id на кожен тиждень (наприклад, сумувати кількість або доходи).

✅ Якщо кожна комбінація з’являється лише один раз, значить, дані вже в нормалізованій формі.

In [ ]:
train_data[train_data["week"] == 1][["meal_id", "center_id", "week"]].groupby(["meal_id", "center_id"]).count()


week
meal_id center_id      
1062    10            1
        11            1
        13            1
        14            1
        17            1
...                 ...
2867    145           1
        146           1
        152           1
        153           1
        174           1

[2922 rows x 1 columns]

✅ Результат:
True — є дублікати meal_id + center_id у тижні 1

False — всі комбінації унікальні (по одному запису кожна)

🔧 Корисно для перевірки цілісності даних перед побудовою моделей чи агрегацією.

In [ ]:
any(train_data[train_data["week"] == 1][["meal_id", "center_id", "week"]].groupby(["meal_id", "center_id"]).count()["week"] > 1)

False